In [15]:
import pickle
import omegaconf
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

project_name = 'full_fine_tuning_50epochs_final1'

datas=( 'uc-merced-land-use-dataset',  'stanford_cars', 'caltech101', 'fgvc-aircraft-2013b', 'dtd', 'flowers-102', 'oxford-iiit-pet'  ) 

losses=( 'TRADES_v2', 'CLASSIC_AT' ) #  

backbones=(
  'convnext_tiny', 'robust_convnext_tiny', 'convnext_tiny.fb_in22k', 
  'deit_small_patch16_224.fb_in1k', 'robust_deit_small_patch16_224',
  'convnext_base', 'convnext_base.fb_in22k', 'robust_convnext_base', 
  'convnext_base.clip_laion2b', 'convnext_base.clip_laion2b_augreg',
  'vit_base_patch16_224.augreg_in1k', 'vit_base_patch16_224.augreg_in21k',
  'vit_base_patch16_224.dino', 'vit_base_patch16_224.mae', 'vit_base_patch16_224.orig_in21k',
  'vit_base_patch16_224.sam_in1k', 'vit_base_patch16_224_miil.in21k', 'robust_vit_base_patch16_224' )  

import pickle
import numpy as np

final_data = []

for backbone in backbones:
    for loss in losses:
        for data in datas:

            try:

                name ='{}_{}_{}'.format(backbone, data, loss)
                with open('./results/results_{}_{}_{}_{}.pkl'.format(project_name, backbone, data, loss), 'rb') as f:
                    saved_data = pickle.load(f)
                    result = saved_data[name]["statistics"]
                    result['backbone'] = backbone
                    result['dataset'] = data
                    result['loss_function'] = loss    

                    print(name, saved_data[name]["statistics"])
            except:
                result = {}
                result['backbone'] = backbone
                result['dataset'] = data
                result['loss_function'] = loss   
                result['clean_acc'] = np.nan
                result['robust_acc'] = np.nan 
                # print('not available')

            final_data.append(result)

convnext_tiny_stanford_cars_TRADES_v2 {'clean_acc': 0.7906666666666666, 'robust_acc': 0.5133333333333333, 'Linf_acc': 0.6246666666666667, 'backbone': 'convnext_tiny', 'dataset': 'stanford_cars', 'loss_function': 'TRADES_v2'}
convnext_tiny_caltech101_TRADES_v2 {'clean_acc': 0.9706666666666667, 'robust_acc': 0.906, 'backbone': 'convnext_tiny', 'dataset': 'caltech101', 'loss_function': 'TRADES_v2'}
convnext_tiny_fgvc-aircraft-2013b_TRADES_v2 {'clean_acc': 0.6653333333333333, 'robust_acc': 0.35733333333333334, 'backbone': 'convnext_tiny', 'dataset': 'fgvc-aircraft-2013b', 'loss_function': 'TRADES_v2'}
convnext_tiny_dtd_TRADES_v2 {'clean_acc': 0.7326666666666667, 'robust_acc': 0.46, 'backbone': 'convnext_tiny', 'dataset': 'dtd', 'loss_function': 'TRADES_v2'}
convnext_tiny_flowers-102_TRADES_v2 {'clean_acc': 0.9553333333333334, 'robust_acc': 0.8153333333333334, 'backbone': 'convnext_tiny', 'dataset': 'flowers-102', 'loss_function': 'TRADES_v2'}
convnext_tiny_oxford-iiit-pet_TRADES_v2 {'clean

In [16]:
import pandas as pd

df = pd.DataFrame(final_data)
df

,backbone,dataset,loss_function,clean_acc,robust_acc,Linf_acc,L2_acc,L1_acc
0,convnext_tiny,uc-merced-land-use-dataset,TRADES_v2,NaN,NaN,NaN,NaN,NaN
1,convnext_tiny,stanford_cars,TRADES_v2,0.790667,0.513333,0.624667,NaN,NaN
2,convnext_tiny,caltech101,TRADES_v2,0.970667,0.906000,NaN,NaN,NaN
3,convnext_tiny,fgvc-aircraft-2013b,TRADES_v2,0.665333,0.357333,NaN,NaN,NaN
4,convnext_tiny,dtd,TRADES_v2,0.732667,0.460000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
247,robust_vit_base_patch16_224,caltech101,CLASSIC_AT,0.193333,NaN,0.189333,NaN,NaN
248,robust_vit_base_patch16_224,fgvc-aircraft-2013b,CLASSIC_AT,0.214667,NaN,0.175333,NaN,NaN
249,robust_vit_base_patch16_224,dtd,CLASSIC_AT,0.268667,NaN,0.197333,NaN,NaN
250,robust_vit_base_patch16_224,flowers-102,CLASSIC_AT,0.006667,NaN,0.002000,NaN,NaN


In [17]:

# Define the valid condition for each row:
#   - either both are not NaN,
#   - or exactly one is NaN (and then the other must be not NaN)
valid_condition = (
    (df['robust_acc'].isna() & df['Linf_acc'].notna()) | 
    (df['Linf_acc'].isna() & df['robust_acc'].notna()) | 
    (df['robust_acc'].isna() & df['Linf_acc'].isna())
)

if valid_condition.all():
    print("Condition holds: If one column is NaN, the other is necessarily not NaN.")
else:
    print("Condition does NOT hold. The following rows do not meet the requirement:")

# set(df[~valid_condition]['dataset'])

df[~valid_condition]


Condition does NOT hold. The following rows do not meet the requirement:


,backbone,dataset,loss_function,clean_acc,robust_acc,Linf_acc,L2_acc,L1_acc
1,convnext_tiny,stanford_cars,TRADES_v2,0.790667,0.513333,0.624667,NaN,NaN
8,convnext_tiny,stanford_cars,CLASSIC_AT,0.692667,0.420000,0.422000,NaN,NaN
15,robust_convnext_tiny,stanford_cars,TRADES_v2,0.758000,0.643333,0.650000,NaN,NaN
22,robust_convnext_tiny,stanford_cars,CLASSIC_AT,0.556667,0.620000,0.414000,NaN,NaN
29,convnext_tiny.fb_in22k,stanford_cars,TRADES_v2,0.799333,0.620667,0.632667,NaN,NaN
36,convnext_tiny.fb_in22k,stanford_cars,CLASSIC_AT,0.591333,0.334000,0.346000,NaN,NaN
90,convnext_base.fb_in22k,oxford-iiit-pet,TRADES_v2,0.875333,0.733333,0.726667,NaN,NaN
230,vit_base_patch16_224_miil.in21k,oxford-iiit-pet,TRADES_v2,0.890667,0.606667,0.627333,NaN,NaN


In [18]:
df[ df['Linf_acc'].notna() ]

,backbone,dataset,loss_function,clean_acc,robust_acc,Linf_acc,L2_acc,L1_acc
1,convnext_tiny,stanford_cars,TRADES_v2,0.790667,0.513333,0.624667,NaN,NaN
6,convnext_tiny,oxford-iiit-pet,TRADES_v2,0.872667,NaN,0.690667,NaN,NaN
8,convnext_tiny,stanford_cars,CLASSIC_AT,0.692667,0.420000,0.422000,NaN,NaN
15,robust_convnext_tiny,stanford_cars,TRADES_v2,0.758000,0.643333,0.650000,NaN,NaN
20,robust_convnext_tiny,oxford-iiit-pet,TRADES_v2,0.854000,NaN,0.763333,NaN,NaN
22,robust_convnext_tiny,stanford_cars,CLASSIC_AT,0.556667,0.620000,0.414000,NaN,NaN
29,convnext_tiny.fb_in22k,stanford_cars,TRADES_v2,0.799333,0.620667,0.632667,NaN,NaN
34,convnext_tiny.fb_in22k,oxford-iiit-pet,TRADES_v2,0.838667,NaN,0.679333,NaN,NaN
36,convnext_tiny.fb_in22k,stanford_cars,CLASSIC_AT,0.591333,0.334000,0.346000,NaN,NaN
48,deit_small_patch16_224.fb_in1k,oxford-iiit-pet,TRADES_v2,0.880667,NaN,0.616667,NaN,NaN


In [19]:
df['Linf_acc'] = df['robust_acc'].combine_first(df['Linf_acc'])
# df['Linf_acc'] = df['Linf_acc'].combine_first(df['robust_acc'])

In [20]:
data_updated = df.copy()

# Combine clean_acc and robust_acc as independent metrics for ranking
data_updated['total_acc'] = data_updated['clean_acc'] + data_updated['Linf_acc']  # Sum of scores per dataset

# Aggregate total_acc across all datasets for each backbone + loss combination
aggregated_data_no_avg = data_updated.groupby(['backbone', 'loss_function']).agg(
    total_sum=('total_acc', 'sum'),  # Total sum of all scores across datasets
    total_geom_mean=('total_acc', lambda x: np.prod(x) ** (1 / len(x))),  # Geometric mean
).reset_index()

# Rank based on these combined scores
aggregated_data_no_avg['rank_total_sum'] = aggregated_data_no_avg['total_sum'].rank(ascending=False)
aggregated_data_no_avg['rank_total_geom_mean'] = aggregated_data_no_avg['total_geom_mean'].rank(ascending=False)

# Sort by sum ranking for presentation
ranked_aggregated_data_no_avg = aggregated_data_no_avg.sort_values(by='rank_total_sum')
ranked_aggregated_data_no_avg

# tools.display_dataframe_to_user(name="Ranked Backbone and Loss Combinations Without Averaging", dataframe=ranked_aggregated_data_no_avg)

,backbone,loss_function,total_sum,total_geom_mean,rank_total_sum,rank_total_geom_mean
1,convnext_base,TRADES_v2,9.540667,1.474117,1.0,1.0
4,convnext_base.clip_laion2b_augreg,CLASSIC_AT,9.298667,1.441563,2.0,2.0
2,convnext_base.clip_laion2b,CLASSIC_AT,9.226000,1.429585,3.0,4.0
11,convnext_tiny.fb_in22k,TRADES_v2,9.211333,1.429607,4.0,3.0
17,robust_convnext_tiny,TRADES_v2,8.998667,1.400416,5.0,5.0
31,vit_base_patch16_224.orig_in21k,TRADES_v2,8.883333,1.379085,6.0,7.0
33,vit_base_patch16_224.sam_in1k,TRADES_v2,8.826667,1.374667,7.0,8.0
0,convnext_base,CLASSIC_AT,8.818000,1.350452,8.0,11.0
6,convnext_base.fb_in22k,CLASSIC_AT,8.730667,1.336765,9.0,13.0
9,convnext_tiny,TRADES_v2,8.730000,1.352120,10.0,10.0


In [21]:
# Pivot the table to group by 'backbone' and 'loss_function' in rows and create a hierarchical structure with 'dataset' as column index

df['total_acc'] = df['clean_acc'] + df['Linf_acc']  # Sum of scores per dataset

grouped_df = df.pivot_table(
    index=['backbone', 'loss_function'], 
    columns='dataset', 
    values=[  'total_acc'  ], #
    dropna=False )

# grouped_df #.to_csv('results_acc.csv' )



# Calculate variance across datasets per row (axis=1)
dataset_variance = grouped_df.var(axis=0)
sorted_variance = dataset_variance.sort_values(ascending=False)
sorted_variance
# Attach to the grouped_df for reference
# grouped_df[('total_acc_variance', '')] = dataset_variance

           dataset                   
total_acc  stanford_cars                 0.202374
           oxford-iiit-pet               0.163573
           caltech101                    0.141293
           flowers-102                   0.138278
           fgvc-aircraft-2013b           0.135654
           dtd                           0.028111
           uc-merced-land-use-dataset         NaN
dtype: float64

In [14]:
           dataset                   
total_acc  flowers-102                   0.250492
           uc-merced-land-use-dataset    0.225154
           stanford_cars                 0.196160
           oxford-iiit-pet               0.156795
           caltech101                    0.126929
           dtd                           0.035525
           fgvc-aircraft-2013b           0.028627
dtype: float64

total_acc            \
dataset                                         caltech101       dtd   
backbone                          loss_function                        
convnext_base                     CLASSIC_AT      1.906667  1.140667   
                                  TRADES_v2       1.899333  1.076000   
convnext_base.clip_laion2b        CLASSIC_AT      1.824667  0.894000   
                                  TRADES_v2       1.788667  0.845333   
convnext_base.clip_laion2b_augreg CLASSIC_AT      1.786667  0.912000   
                                  TRADES_v2       1.784000  0.814000   
convnext_base.fb_in22k            CLASSIC_AT      1.911333  0.965833   
                                  TRADES_v2       1.906667  1.016667   
convnext_tiny                     CLASSIC_AT      1.811333  0.569500   
                                  TRADES_v2       1.754000  0.580833   
convnext_tiny.fb_in22k            CLASSIC_AT      1.862000  0.944000   
                                  TRADES_v2       1.859333  0.818667   
deit_small_patch16_224.fb_in1k    CLASSIC_AT      1.790667  0.898667   
                                  TRADES_v2       1.766000  0.832000   
robust_convnext_base              CLASSIC_AT      1.881333  0.906667   
                                  TRADES_v2       1.897333  0.800000   
robust_convnext_tiny              CLASSIC_AT      1.884000  0.877333   
                                  TRADES_v2       1.854667  0.720667   
robust_deit_small_patch16_224     CLASSIC_AT      1.866667  0.829333   
                                  TRADES_v2       1.849333  0.862000   
robust_vit_base_patch16_224       CLASSIC_AT      0.160667  0.290667   
                                  TRADES_v2       1.308667  0.556667   
vit_base_patch16_224.augreg_in1k  CLASSIC_AT      1.741333  0.820000   
                                  TRADES_v2       1.740667  0.774000   
vit_base_patch16_224.augreg_in21k CLASSIC_AT      1.712667  0.807667   
                                  TRADES_v2       1.808000  0.463500   
vit_base_patch16_224.dino         CLASSIC_AT      1.795333  0.880667   
                                  TRADES_v2       1.755333  0.796667   
vit_base_patch16_224.mae          CLASSIC_AT      1.501333  0.510667   
                                  TRADES_v2       1.493333  0.475333   
vit_base_patch16_224.orig_in21k   CLASSIC_AT      1.681333  0.924667   
                                  TRADES_v2       1.760000  0.900000   
vit_base_patch16_224.sam_in1k     CLASSIC_AT      1.738667  0.862000   
                                  TRADES_v2       1.796667  0.890000   
vit_base_patch16_224_miil.in21k   CLASSIC_AT      0.579333  0.517333   
                                  TRADES_v2       1.425333  0.573333   

                                                                     \
dataset                                         fgvc-aircraft-2013b   
backbone                          loss_function                       
convnext_base                     CLASSIC_AT               0.123833   
                                  TRADES_v2                0.337333   
convnext_base.clip_laion2b        CLASSIC_AT               0.404667   
                                  TRADES_v2                0.644667   
convnext_base.clip_laion2b_augreg CLASSIC_AT               0.396667   
                                  TRADES_v2                0.436667   
convnext_base.fb_in22k            CLASSIC_AT               0.218667   
                                  TRADES_v2                0.560000   
convnext_tiny                     CLASSIC_AT               0.063333   
                                  TRADES_v2                0.145333   
convnext_tiny.fb_in22k            CLASSIC_AT               0.188667   
                                  TRADES_v2                0.498000   
deit_small_patch16_224.fb_in1k    CLASSIC_AT               0.084000   
                                  TRADES_v2                0.130000   
robust_convnext_base             

In [26]:
import pandas as pd
import numpy as np
import plotly.express as px


df_new_cleaned = df #.drop(columns=['Unnamed: 0'])
name =  'TRADES_v2' #
df_new_cleaned = df_new_cleaned[ df_new_cleaned.loss_function != name ]

# Assuming df_new_cleaned already exists and contains all necessary columns
# Group by backbone and loss function, compute the mean and standard deviation for clean and robust accuracy
aggregated_metrics = df_new_cleaned.groupby(['backbone', 'loss_function']).agg(
    Mean_Clean_Acc=('clean_acc', 'mean'),
    Std_Clean_Acc=('clean_acc', 'std'),
    Mean_Robust_Acc=('robust_acc', 'mean'),
    Std_Robust_Acc=('robust_acc', 'std')
).reset_index()

# Classify architectures into categories
aggregated_metrics['Architecture'] = aggregated_metrics['backbone'].apply(
    lambda name: 'ConvNext' if 'convnext' in name.lower() else ('ViT/DeiT' if 'deit' in name.lower() or 'vit' in name.lower() else 'Other')
)

# Assign marker shapes based on loss function
aggregated_metrics['Marker_Shape'] = aggregated_metrics['loss_function'].apply(
    lambda loss: 'circle' if loss == 'TRADES_v2' else ('square' if loss == 'CLASSIC_AT' else 'diamond')
)

# Identify Pareto front
x = aggregated_metrics['Mean_Clean_Acc']
y = aggregated_metrics['Mean_Robust_Acc']
is_pareto = np.array([
    not any((y[j] >= y[i] and x[j] >= x[i]) for j in range(len(x)) if i != j)
    for i in range(len(x))
])
pareto_points = aggregated_metrics[is_pareto]

# Add a mapping for loss functions to valid Plotly symbols
symbol_mapping = {
    'TRADES_v2': 'circle',
    'CLASSIC_AT': 'square'
}

# Update the scatter plot with proper legend entries for loss functions
fig = px.scatter(
    aggregated_metrics,
    x='Mean_Clean_Acc',
    y='Mean_Robust_Acc',
    color='Architecture',         # Color for architecture
    symbol='loss_function',       # Symbol for loss function
    # title='Backbone Performance: Clean vs. Robust Accuracy',
    labels={
        'Mean_Clean_Acc': 'Mean Clean Accuracy',
        'Mean_Robust_Acc': 'Mean Robust Accuracy',
        'loss_function': 'Loss Function'  # Custom label for legend
    },
    color_discrete_map={
        'ConvNext': '#1f77b4',  # Blue for ConvNext
        'ViT/DeiT': '#ff7f0e',  # Orange for ViT/DeiT
        'Other': '#2ca02c'      # Green for Other architectures
    },
    symbol_map=symbol_mapping    # Apply symbol mapping
)

# Add static backbone names as labels on the figure
for i, row in aggregated_metrics.iterrows():
    fig.add_annotation(
        x=row['Mean_Clean_Acc'] - 0.01,
        y=row['Mean_Robust_Acc'] + 0.01,  # Offset text slightly above the marker
        text=row['backbone'],
        showarrow=False,
        font=dict(size=10),
        align='center'
    )

# Add Pareto front markers with larger size and same shape/color as the original
for idx, d in enumerate(pareto_points.iterrows()):
    _, row = d
    fig.add_scatter(
        x=[row['Mean_Clean_Acc']],
        y=[row['Mean_Robust_Acc']],
        mode='markers',
        showlegend=False,  # Avoid duplicating legends
        marker=dict(
            size=13,  # Larger marker size
            symbol=symbol_mapping[row['loss_function']],  # Use the mapped symbol
            color=aggregated_metrics.loc[aggregated_metrics['backbone'] == row['backbone'], 'Architecture'].map({
                'ConvNext': '#1f77b4',  # Blue for ConvNext
                'ViT/DeiT': '#ff7f0e',  # Orange for ViT/DeiT
                'Other': '#2ca02c'     # Green for Other architectures
            }).values[0]  # Map the color dynamically
        ),
        name=f"Pareto"
    )


# Add a diagonal line (x = y)
fig.add_shape(
    type="line",
    x0=0, y0=0,  # Start of line
    x1=1, y1=1,  # End of line
    xref="paper", yref="paper",  # Use normalized coordinates (0 to 1 in plot space)
    line=dict(color="red", width=2, dash="dot")  # Line style
)

# Adjust layout for better visual clarity
fig.update_layout(
    legend_title=dict(text='Backbone and Loss Function'),
    width=600,
    height=600,
    font=dict(size=12),
    #title_font=dict(size=14),
    margin=dict(l=10, r=45, t=10, b=10),
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    xaxis=dict(showgrid=True, gridcolor='lightgrey', zeroline=False),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', zeroline=False),
    yaxis_range=[0,0.7],
    xaxis_range=[0,0.85]
)


fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85 ],
        ticktext=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85 ]
    ),
    yaxis=dict(
        tickmode="array",
        tickvals=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7 ],
        ticktext=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7 ]
    )
)

fig.show()
fig.write_image( "./figures/{}_{}.png".format(project_name,name) )